In [85]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from time import sleep
import tqdm
import json
import time

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from selenium.webdriver.common.by import By

In [86]:
import warnings
warnings.filterwarnings('ignore')

## 1. Парсинг URL адресов с новостями (Selenium)

In [87]:
#Download driver using the link at the top of the notebook
chrome_driver_path = r"chromedriver_win32\chromedriver.exe"
links_filename = "links.txt"

In [100]:
categories = ["politic", "society", "accident", "conflict"]
cat_translate = {"Политика":"politic", 
                 "Общество":"society", 
                 'Происшествия':"accident", 
                 "Конфликты":"conflict",
                 'Преступность':"crime"
                 }


In [89]:
# установка ublock чтобы это расширение было установлено по умолчанию

# options = Options()
# options.add_argument("user-data-dir=C:\\profile")
# driver = Chrome(chrome_options=options)
# driver.get("https://google.com")

In [90]:
URL = r"https://lenta.ru/rubrics/world/"
start_time = time.time()

def connect():
    chrome_options = Options()
    chrome_options.add_argument("user-data-dir=C:\\profile")
    chrome_options.add_argument('--disable-application-cache')
    driver = Chrome(executable_path=chrome_driver_path, chrome_options=chrome_options)
    driver.implicitly_wait(5)
    
    print(f"Current session is {driver.session_id}")
    # print("now time:", time.time()-start_time)
    # driver.get(url)
    # time.sleep(3)

    return driver


def get_links_from_one_page(driver, url):
    driver.get(url)
    time.sleep(5)
    
    recipe_links = []
    for i in range(1, 3):
        a = driver.find_element(By.XPATH, f"/html/body/div[3]/div[3]/main/div/section/ul/li[{i}]/a")
        # print("now time:", time.time()-start_time)
        recipe_links.append(a.get_attribute("href"))
    return recipe_links


driver = connect()

for category in categories:
    links = []
    
    for page in range(1, 3):
        
        # print("now time1:", time.time() - start_time)
        links.extend(get_links_from_one_page(driver, URL + category + f"/{page}/"))
        # print("now time2:", time.time() - start_time)
        # driver.close()
        # print("now time3:", time.time() - start_time)
        # print(links)

    with open(links_filename, "a") as file:
        for link in links:
            file.write(link + "\n")

Current session is d1a0bc69a807cf0f029ae73d59cb6465


## 2. Парсинг новостей (BeautifulSoup)

In [91]:
posts = []
with open(links_filename) as file:
    for line in file:
        posts.append(line[:-1])

In [92]:
posts[0]

'https://lenta.ru/news/2023/01/17/vccvk/'

In [93]:
df = pd.DataFrame(columns=["link", "category", "title", "text"])

In [101]:
index = 0 

for post in posts:
    resp = requests.get(post)
    soup = BeautifulSoup(resp.text, 'lxml')
    
    title = soup.title.text
    sep_position = title.find(":") 
    category = title[sep_position+2 : title.find(":", sep_position+1)]
    category = cat_translate[category]
    title = title[ : sep_position]

    text = soup.find("div", class_ = "topic-body__content").get_text()

    df.loc[index] = [post, category, title, text] 
    index += 1

'accident'

## 3. Обзор полученных данных

In [103]:
df.describe()

,link,category,title,text
count,286,286,286,286
unique,236,5,236,236
top,https://lenta.ru/news/2023/01/17/zaderzhan/,politic,Проигравшего на выборах республиканца задержал...,Полиция города Альбукерке в Штате Нью-Мексико ...
freq,4,149,4,4


In [102]:
df.head()

,link,category,title,text
0,https://lenta.ru/news/2023/01/17/vccvk/,politic,Вучич осудил контакты сербских активистов с ЧВК,Президент Сербии Александр Вучич осудил контак...
1,https://lenta.ru/news/2023/01/17/strategy/,politic,В США заявили о кровавой стратегии Вашингтона ...,Соединенные Штаты используют Украину как пешку...
2,https://lenta.ru/news/2023/01/17/tur_swe/,politic,Турция назвала условие для вступления Швеции в...,Швеция не может ожидать поддержки Турции в воп...
3,https://lenta.ru/news/2023/01/16/pomni_imya_svoe/,politic,Байден забыл имя юбилярши и выкрутился,Президент США Джо Байден забыл имя правозащитн...
4,https://lenta.ru/news/2023/01/17/proof/,politic,Суд в Италии запросил у США дополнительные док...,Суд в Италии запросил у США дополнительные док...


## 4. Конвертирование в JSON формат

In [104]:
js = json.loads(df.to_json(force_ascii=False, orient='records'))

In [105]:
with open('corpus.json', 'w') as file:
    for i in js:
        file.write(json.dumps(i) + '\n')
